In [1]:
import pandas as pd
from datetime import datetime
# Headers and datatypes for pandas dframe
header_raw = (r'^ID;Timestamp;GyroscopeX;GyroscopeY;GyroscopeZ;AccelerometerX;AccelerometerY;' \
           'AccelerometerZ;Barometer;Thermometer;ThermometerStupido;Voltage$')

headers = [
    "ID", "Timestamp", "GyroscopeX", "GyroscopeY", "GyroscopeZ", "AccelerometerX", "AccelerometerY",
    "AccelerometerZ", "Barometer", "Thermometer", "ThermometerStupido", "Voltage",
]

datatypes = [int, int, float, float, float, float, float, float, int, float, float, float]


In [2]:
def init_dataframe(data=None):
    if data == None:
        return pd.DataFrame({}, columns=headers)
    return pd.DataFrame(data, columns=headers, index=[0])

def dump_dataframe(df):
    df.to_csv(f'LOG_{datetime.now().isoformat()}.csv', index=False)

def is_csv_header(line):
    # Simple heuristic to check if a line is a CSV header
    return bool(re.match(header_raw, line))

def is_csv_row(line):
    # Simple heuristic to check if a line could be a CSV row
    return bool(re.match(r'^-?\d+(\.\d+)?(;-?\d+(\.\d+)?)*$', line))

def append_csv_row(row, df):
    values = row.split(';')
    data = {header: dt(val) for header, dt, val in zip(headers, datatypes, values)}
    return pd.concat((df, init_dataframe(data=data)), ignore_index=True).sort_values('ID')

def process_line(line, df):
    if is_csv_header(line):
        dump_dataframe(df)
    elif is_csv_row(line):
        append_csv_row(line, df)
        

In [3]:
import sys
import re

import pandas as pd
import serial

# Expected indexes of gyro and acc
gyro = [2, 3, 4]
aclr = [5, 6, 7]

# Activates when CSV header is read

device = '/dev/ttyACM0'
ser = serial.Serial(device, 115200, timeout=60*60)
line = b''

df = init_dataframe()
activated = False

while x := ser.read(1):
    if x[0] == 10:
        try:
            line = line.decode("UTF-8").strip()
            if is_csv_header(line):
                if activated:
                    dump_dataframe(df)
                    df = init_dataframe()
                activated = True
            elif is_csv_row(line) and activated:
                df = append_csv_row(line, df)
            else:
                print(repr(line))
        except Exception as e:
            print(e)
            print(f"Error {activated}: {line}")
        finally:
            line = b''
        
    else:
        line += x
    

'0;0.00;0.00;0.00;0.00;7;8.00;9.00;10.00'
'utf-8' codec can't decode byte 0xf4 in position 1: invalid continuation byte
Error False: b"e\xf4.R\xd5n\xa0\xc2\x02\x920\xb1\x13\x82\xc2:&\xa6\xaa7\r\xa1HH\xcd\xd1i0\xf8LB\x02=]ER\xca}\xc9E\xd3QU\x89\t\xbdot'\xc2E3 i\x05\x95}\x19\x85S\x94e1AS\xea%\xbdO\xd4JC!\x96+\xcd\xa1\x01reX\x16*\xc9\xc9q \xb1\x82\x82*"
'utf-8' codec can't decode byte 0xfd in position 2: invalid start byte
Error False: b'YW\xfdm\xe1Z\xcbi\x81\xb3S\x02*'
'utf-8' codec can't decode byte 0xf4 in position 1: invalid continuation byte
Error False: b"e\xf4.R\xd5\xb9\x81 8\x92\x82\x8a6\xa0\x82\xd2\x922\xbaMSH\r!\xae\xd1:\xb0\xc5\xb0B\x92Q\r]DT\xa5Q\x8d_\xd2Qj\x14U\x89I\xbd\xbdt:\xc2\xc53 \xa8*\x95}\x19\x05STe1\x85S\xc8%=\xbdT\xa9C\xa1\xac\xcb\x99ig\xae\x0bi \xb0\t\x9aA%]P:\xc2\x95\x95\r\x8a,k\xebE\xc9v:\xc2\xc10,\xf1\x05\xc9\xd9:\xb0\xc1\xb0\x89}dr\x97\x82\xc2\xc1\xb0)\xcd0_\x96\xd9i0\xf8\x82b\xc2\x91}dr\xb6'\xc2\xc10i\xc1\xbdd\xf2\x17\x82\xc2\xc10\r"
'utf-8' codec can't decode 

/tmp/ipykernel_179053/3665151942.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat((df, init_dataframe(data=data)), ignore_index=True).sort_values('ID')


'Received msg: 1174'
'Received msg: 1170'
'Received msg: 1171'
'Received msg: 1172'
'Received msg: 1173'
'Received msg: 1175'
'Received msg: 1170'
'Received msg: 1176'
'Received msg: 1177'
'Received msg: 1178'
'Received msg: 1179'
'Received msg: 1180'
'Received msg: 1181'
'Received msg: 1182'
'Received msg: 1183'
'Received msg: 1184'
'Received msg: 1185'
'Received msg: 1186'
'Received msg: 1187'


KeyboardInterrupt: 

In [ ]:
ord('\n')

In [ ]:
header_raw

In [ ]:
dir(df)

In [ ]:
'ID;Timestamp;GyroscopeX;GyroscopeY;GyroscopeZ;AccelerometerX;AccelerometerY;AccelerometerZ;Barometer;Thermometer;ThermometerStupido;Voltage\r'.strip()

In [ ]:
df._append